In [100]:
from __future__ import division, print_function

import collections
import cPickle as pickle

import numpy as np

from astropy import units as u
from astropy.io import fits
from astropy.table import Table
from astropy import wcs

In [21]:
drz_platescale = 0.03*u.arcsec #per pixel

# Load Data 

In [2]:
def unpickle_file(fn):
    with open(fn, 'r') as f:
        return pickle.load(f)

In [12]:
struc_param_dct = unpickle_file('strucparams_chains_2015_07_22.pckl')

def get_struc_param(chainnm, paramnm):
    sp = struc_param_dct['chains'+chainnm]
    idx = struc_param_dct['param_names'].index(paramnm)
    return sp[:, :, idx].ravel()

struc_param_dct.keys(), struc_param_dct['param_names']

(['param_names', 'chains606B', 'chains814B', 'chains814A', 'chains606A'],
 ['flux', 'n', 'reffmaj', 'ellipticity', 'cenx', 'ceny', 'theta'])

In [65]:
dists = np.load('PiscAB_distances_2015_12_15.npz')
#rgbchains = np.load('rgb_chains_2015_12_15.npz')
rgbchains = np.load('rgb_chains_2015_12_23.npz') # modA/modB are missing because this one was reprocessed

def get_rgb_param(aorb, paramnm):
    if aorb == 'A':
        chain = rgbchains['chainsA']
    elif aorb == 'B':
        chain = rgbchains['chainsB']
    else:
        raise ValueError('not A or B')
    
    idx = list(rgbchains['param_names']).index(paramnm)
    return chain[:, :, idx].ravel()

dists.keys(), rgbchains.keys(), rgbchains['param_names']

(['piscA_dist_Mpc', 'piscB_dist_Mpc'],
 ['tipcolorA', 'param_names', 'tipcolorB', 'chainsA', 'chainsB'],
 array(['tipmag', 'alphargb', 'alphaother', 'fracother'], 
       dtype='|S10'))

# Process data into table 

In [77]:
f606wzpt = 26.407*u.mag
f814wzpt = 25.523*u.mag

def get_flux_mag(aorb, band):
    band = str(band)
    
    if aorb == 'A':
        flux = get_struc_param(band+'A', 'flux')
    elif aorb == 'B':
        flux = get_struc_param(band+'B', 'flux')
    else:
        raise ValueError('not A or B')
        
    if band=='606':
        zpt = f606wzpt
    elif band=='814':
        zpt = f814wzpt
    else:
        raise ValueError('invalid band')
        
    return  zpt - 2.5*np.log10(flux)*u.mag

In [96]:
def get_phys_PA(aorb, band):
    band = str(band)
    
    if aorb == 'A':
        theta = get_struc_param(band+'A', 'theta')
        wcsfn = 'drizzled_PiscA_F606W_err/PiscA_F606W_err_drc_sci.fits'
    elif aorb == 'B':
        theta = get_struc_param(band+'B', 'theta')
        wcsfn = 'drizzled_PiscB_F606W_err/PiscB_F606W_err_drc_sci.fits'
    else:
        raise ValueError('not A or B')
        
    w = wcs.WCS(fits.getheader(wcsfn))
    
    return  theta*u.deg + np.arctan2(*w.pixel_scale_matrix[0])*u.rad

In [7]:
def minify_chains(*chains):
    chains = [np.array(chain, copy=False).ravel() for chain in chains]
    minchain = np.min([chain.size for chain in chains])
    return [chain[:minchain] for chain in chains]

* Distance
* F814W magnitude of the TRGB 
* (F606W − F814W)0 color of the TRGB
* α
* β
* f (from Equation 1)
* F606W (total)
* F814W (total)
* Sersic index (F606W)
* ellipticity (F606W)
* position angle (F606W)
* Reff (on-sky half-light radius in F606W)
* reff (physical)

In [107]:
tab = Table()

for aorb in ['A', 'B']:
    chains = collections.OrderedDict()
    chains['Distance'] = dists['pisc{}_dist_Mpc'.format(aorb)].ravel()*u.Mpc
    chains['mu_814'] = get_rgb_param(aorb, 'tipmag') * u.mag
    chains['Color_TRGB'] = rgbchains['tipcolor'+aorb].ravel() * u.mag
    chains['alpha'] = get_rgb_param(aorb, 'alphargb')
    chains['beta'] = get_rgb_param(aorb, 'alphaother')
    chains['f'] = get_rgb_param(aorb, 'fracother')
    chains['F606W'] = get_flux_mag(aorb, 606)
    chains['F814W'] = get_flux_mag(aorb, 814)
    chains['sersic_n'] = get_struc_param('606' + aorb, 'n')
    chains['e'] = get_struc_param('606' + aorb, 'ellipticity')
    chains['PA'] = get_phys_PA(aorb, 606)
    chains['Reff_major'] = get_struc_param('606' + aorb, 'reffmaj')*drz_platescale

    minchains = minify_chains(*chains.values())
    
    chains2 = collections.OrderedDict()
    for i, (k, v) in enumerate(chains.items()):
        if hasattr(v, 'unit'):
            chains2[k] = minchains[i]*v.unit
        else:
            chains2[k] = minchains[i]

    chains2['reff'] = (chains2['Reff_major'].to(u.radian).value * chains2['Distance'] * chains2['e']**0.5 ).to(u.pc)
    
    for k, v in chains2.items():
        tab[k+'_'+aorb] = v

In [121]:
tab

Distance_A,mu_814_A,Color_TRGB_A,alpha_A,beta_A,f_A,F606W_A,F814W_A,sersic_n_A,e_A,PA_A,Reff_major_A,reff_A,Distance_B,mu_814_B,Color_TRGB_B,alpha_B,beta_B,f_B,F606W_B,F814W_B,sersic_n_B,e_B,PA_B,Reff_major_B,reff_B
Mpc,mag,mag,,,,mag,mag,,,deg,arcsec,pc,Mpc,mag,mag,,,,mag,mag,,,deg,arcsec,pc
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
5.35137251305,24.5061705213,0.849223016923,0.651241787818,0.650790579957,0.299216478809,17.3217352382,16.6033721723,0.44769370305,0.345743045354,135.03480639,9.03179510149,137.781482509,8.281959086,25.4635393262,0.894369613673,0.412747481966,2.74093636118,0.621478009672,16.9638911081,16.4184332754,0.645326072171,0.523676123176,139.143872298,10.3846524048,301.73889693
5.33509409207,24.4996550196,0.849723016923,0.660155648614,0.458199892511,0.198218745088,17.3212153282,16.6035109632,0.450046928886,0.347035301271,135.096933126,9.05296541316,137.941402872,8.281959086,25.4635393262,0.894369613673,0.412747481966,2.74093636118,0.621478009672,16.9638911081,16.4184332754,0.645326072171,0.523676123176,139.143872298,10.3846524048,301.73889693
5.33509409207,24.4996550196,0.849723016923,0.660155648614,0.458199892511,0.198218745088,17.3202196011,16.6035109632,0.451696853959,0.347765658512,134.620767985,9.03935270764,137.878842685,8.07932977429,25.4064505983,0.877869613673,0.487071813201,2.72338503647,0.583123585626,16.9638911081,16.4184332754,0.645326072171,0.523676123176,139.143872298,10.3846524048,294.356447395
5.42143314792,24.5343151375,0.848723016923,0.564982978913,0.536925192628,0.214519180084,17.3218216991,16.6035109632,0.458506653795,0.339294018433,135.366458305,8.99808624221,137.761298182,8.49856436973,25.5151017637,0.871869613673,0.34561787832,2.75678868642,0.656119657186,16.9636221964,16.4184332754,0.645895728694,0.524071505863,139.138494254,10.3879673389,309.846278446
5.42143314792,24.5343151375,0.848723016923,0.564982978913,0.536925192628,0.214519180084,17.3218216991,16.60208575,0.458506653795,0.339294018433,135.366458305,8.99808624221,137.761298182,8.49793094169,25.5158399101,0.876369613673,0.345066710216,2.75574137778,0.656497092218,16.9636221964,16.4203264035,0.645895728694,0.524071505863,139.138494254,10.3879673389,309.823184508
5.42143314792,24.5343151375,0.848723016923,0.564982978913,0.536925192628,0.214519180084,17.3222703421,16.60208575,0.458533958403,0.338244421697,135.484905774,8.99289438148,137.468688048,8.73323146778,25.5721487775,0.861369613673,0.464398364375,2.61032107748,0.72007509027,16.9636221964,16.4167832526,0.645895728694,0.524071505863,139.138494254,10.3879673389,318.401926652
5.50962461172,24.5678546534,0.841223016923,0.535194814384,0.821538842039,0.329314758791,17.3241401527,16.6018691526,0.462600434366,0.338917524954,135.852646522,9.00007365691,139.955494686,8.73323146778,25.5721487775,0.861369613673,0.464398364375,2.61032107748,0.72007509027,16.9636221964,16.4167832526,0.645895728694,0.524071505863,139.138494254,10.3879673389,318.401926652
5.50862559965,24.5674608835,0.841223016923,0.534672580642,0.81803525658,0.326855882291,17.3241401527,16.6018691526,0.462600434366,0.338917524954,135.852646522,9.00007365691,139.93011778,8.73323146778,25.5721487775,0.861369613673,0.464398364375,2.61032107748,0.72007509027,16.9649450257,16.4167832526,0.644795963233,0.52539308201,139.139253914,10.3920623424,318.928812793


In [123]:
tab.write('tab2.ecsv', format='ascii.ecsv')